In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout


In [ ]:
le = LabelEncoder()
images = []
labels = []

training_paths = ['data/raw/character_set1/training_data/',
                 'data/raw/character_set3/', ]
#testing_path = 'data/raw/character_set1/testing_data/'


In [ ]:
for training_path in training_paths:
    dir_list = os.listdir(training_path)
    for i in dir_list:
        print(f"Processing label: {i}")  # Add this line to check labels
        dir = os.path.join(training_path, i)
        file_list = os.listdir(dir)
        for j in file_list:
            files = os.path.join(dir, j)
            img = cv2.imread(files, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Warning: Unable to read image {files}. Skipping.")
                continue
            img = cv2.resize(img, (64, 64))
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            img = img.astype('float32') / 255.0
            images.append(img)
            labels.append(i)

## Print out details of X and y

In [ ]:
X = np.array(images)
print("len(X): ",len(X))
print("X.shape: ", X.shape)

y = np.array(labels)
print("len(y): ",len(y))
print("y.shape: ", y.shape)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
print("Classes:", le.classes_)
X_sh, y_sh = shuffle(X, y, random_state=42)

In [ ]:
# Split the data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X_sh, y_sh, test_size=0.2, random_state=42, stratify=y_sh
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

# Create Keras model
create a neural network model

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(units=256, activation='relu'),
    Dropout(0.5),
    Dense(units=len(le.classes_), activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Fit the data generator
datagen.fit(X_train)

# Use the generator in model training
history = model.fit(datagen.flow(X_train, y_train, batch_size=25),
                    validation_data=(X_test, y_test), epochs=20)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Separate Evaluation for Uppercase and Lowercase Characters

# Determine the number of uppercase letters
# This assumes that the first 'num_uppercase' classes are uppercase letters
# Adjust 'num_uppercase' based on your actual label encoding
num_uppercase = 26  # Typically, A-Z

# Find indices for uppercase and lowercase characters
uppercase_indices = np.where(y_test < num_uppercase)
lowercase_indices = np.where(y_test >= num_uppercase)

# Evaluate the model on uppercase characters
test_loss_upper, test_acc_upper = model.evaluate(X_test[uppercase_indices], y_test[uppercase_indices], verbose=0)
print(f"Uppercase Test Loss: {test_loss_upper:.4f}")
print(f"Uppercase Test Accuracy: {test_acc_upper:.4f}")

# Evaluate the model on lowercase characters
test_loss_lower, test_acc_lower = model.evaluate(X_test[lowercase_indices], y_test[lowercase_indices], verbose=0)
print(f"Lowercase Test Loss: {test_loss_lower:.4f}")
print(f"Lowercase Test Accuracy: {test_acc_lower:.4f}")



In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

# Plot training & validation accuracy values
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


## Visualization

### Visualize Regions

In [ ]:
def visualize_regions(image, regions):
    debug_image = image.copy()
    for (x, y, w, h) in regions:
        cv2.rectangle(debug_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow("Detected Text Regions", debug_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Visualize Processed Image

In [ ]:
def display_processed_image(window_name, processed_image):
    cv2.imshow(window_name, processed_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Image Processing Operations

In [ ]:
def image_processing_operations_visualization(image, operation):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    if operation == 'threshold':
        # Apply thresholding to get a binary image
        _, processed_image = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        window_name = "Thresholded Image"

    elif operation == 'erosion':
        # Apply thresholding and then erosion
        _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        kernel = np.ones((2, 1), np.uint8)
        processed_image = cv2.erode(thresh, kernel, iterations=1)
        window_name = "Eroded Image"

    elif operation == 'dilation':
        # Apply thresholding and then dilation
        _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        kernel = np.ones((1, 1), np.uint8)
        processed_image = cv2.dilate(thresh, kernel, iterations=1)
        window_name = "Dilated Image"

    else:
        raise ValueError("Operation must be 'threshold', 'erosion', or 'dilation'")

    # Display the processed image
    display_processed_image(window_name, processed_image)
    
    return _, processed_image

## Italic Characters Detection

### Compute Skew Angle

In [ ]:
def compute_skew_angle(region):
    # Use moments to calculate the skew angle of a text region
    coords = np.column_stack(np.where(region > 0))
    rect = cv2.minAreaRect(coords)
    angle = rect[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    return angle

### Deskew Angle

In [ ]:
def deskew_region(region, angle):
    # Rotate the region by the computed angle
    (h, w) = region.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(region, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    return rotated

## Detect Text Regions

In [ ]:
def detect_text_regions(image):
    # Convert image to grayscale if it's not already
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Apply adaptive thresholding with adjusted parameters
    processed_image = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 15, 10  # Increased block size and constant
    )
    
    # Find contours
    contours, _ = cv2.findContours(
        processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    regions = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Adjusted size thresholds
        if w > 3 and h > 3:
            regions.append((x, y, w, h))
    return regions

## Sorting Bounding Boxes by Rows and Columns

In [ ]:
def sort_bounding_boxes(regions):
    # Sort by `y` first (top-to-bottom) with a threshold to group by rows
    row_threshold = 20  # Adjust based on character spacing
    regions = sorted(regions, key=lambda box: box[1])

    # Group bounding boxes into rows
    rows = []
    current_row = [regions[0]]
    
    for i in range(1, len(regions)):
        if abs(regions[i][1] - current_row[-1][1]) < row_threshold:
            current_row.append(regions[i])
        else:
            rows.append(current_row)
            current_row = [regions[i]]
    rows.append(current_row)
    
    # Sort each row left-to-right
    sorted_regions = []
    for row in rows:
        sorted_row = sorted(row, key=lambda box: box[0])
        sorted_regions.extend(sorted_row)
    
    return sorted_regions


## Resize Image Operation

In [ ]:
def resize_image(image, target_size=(64, 64)):
    # Resize while maintaining aspect ratio and pad the image
    (h, w) = image.shape[:2]
    scaling_factor = min(target_size[0]/w, target_size[1]/h)
    new_size = (int(w * scaling_factor), int(h * scaling_factor))
    resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    
    # Pad the resized image to make it target_size
    delta_w = target_size[0] - new_size[0]
    delta_h = target_size[1] - new_size[1]
    top, bottom = delta_h//2, delta_h - (delta_h//2)
    left, right = delta_w//2, delta_w - (delta_w//2)
    padded_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=255
    )
    return padded_image

## Recognize Characters

In [ ]:
def recognize_characters(image, model, label_encoder):
    regions = detect_text_regions(image)
    sorted_regions = sort_bounding_boxes(regions)
    characters = []
    for (x, y, w, h) in sorted_regions:
        char_image = image[y:y+h, x:x+w]
        # Convert to grayscale if necessary
        if len(char_image.shape) == 3:
            char_image = cv2.cvtColor(char_image, cv2.COLOR_BGR2GRAY)
        # Deskew the character image
        angle = compute_skew_angle(char_image)
        char_image = deskew_region(char_image, angle)
        # Resize and normalize
        char_image_resized = resize_image(char_image, (64, 64))
        char_image_normalized = char_image_resized.astype('float32') / 255.0
        # Expand dimensions to match input shape
        char_image_normalized = np.expand_dims(char_image_normalized, axis=-1)
        char_image_normalized = np.expand_dims(char_image_normalized, axis=0)
        # Predict
        prediction = model.predict(char_image_normalized)
        predicted_class = np.argmax(prediction, axis=1)
        predicted_char = label_encoder.inverse_transform(predicted_class)[0]
        characters.append(predicted_char)
    return characters

# Driver Code

In [ ]:
imggg = 'data/raw/character_set1/Test_1.png'
image = cv2.imread(imggg) 
y_pred = recognize_characters(image, model, le)
print(y_pred)
print(len(y_pred))
print(''.join(y_pred))
print('IntroductiontoAlgorithmsFourthEdition')


# Load a single lowercase character image
test_img = cv2.imread('data/raw/character_set3/a/A_L_1.png', cv2.IMREAD_GRAYSCALE)
test_img_resized = cv2.resize(test_img, (64, 64)).astype('float32') / 255.0
test_img_input = np.expand_dims(np.expand_dims(test_img_resized, axis=-1), axis=0)

prediction = model.predict(test_img_input)
predicted_class = np.argmax(prediction, axis=1)
predicted_char = le.inverse_transform(predicted_class)
print("Predicted character:", predicted_char)
